In [0]:
%pip install -r requirements_base.txt

%restart_python

In [0]:
%load_ext autoreload
%autoreload 2 

In [0]:
#url = "https://docs.google.com/spreadsheets/d/1PK2eEqNqX276YBgOOWz7MHWpWXAyKo5J1Bil7BgmsCs/edit?gid=0#gid=0"

#languages = "Spanish (Spain), German"

In [0]:
import pandas as pd
import json
import random
import openai
from openai import OpenAI
from pathlib import Path
from unidecode import unidecode
import gspread
import os


import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *

import time
import datetime
import re
import tiktoken

from bert_score import score as bert_score


import mlflow

In [0]:
%run "./setup"

In [0]:
#from ProjectConfigs.aso_monthly_config import LANG_MAP, get_processing_rows_from_sheet_aso_monthly, get_row_dicts

In [0]:
%run "./authenticationScript"

In [0]:
gc = get_gspread_client_from_secret()

def get_processing_rows_from_sheet_aso_monthly():

    #WORKBOOK_ID = "1WnCBGk1V1nEUeU7aPWBqrQXczmaMop6AbGmkDKkAQcE"
    ASO_MONTHLY_SS_URL = "https://docs.google.com/spreadsheets/d/1WnCBGk1V1nEUeU7aPWBqrQXczmaMop6AbGmkDKkAQcE/edit?gid=1033119277#gid=1033119277"

    WKSHT_NAME = "Validation and Url Tracking"

    source_ss = gc.open_by_url(ASO_MONTHLY_SS_URL)

    source_wksht = source_ss.worksheet(WKSHT_NAME)
    vals = source_wksht.get_all_values()

    headers, data = vals[0],vals[1:]
    df = pd.DataFrame(data, columns=headers)

    to_process_df = df[df.JobStatus == 'NotStarted']


    return to_process_df


to_process_df = get_processing_rows_from_sheet_aso_monthly()

def get_row_dicts(to_process_df:pd.DataFrame, 
                  source_cd:str = 'ASO Monthly')->list:

    list_slugs = []
    for i,row in to_process_df.iterrows():
        row_dict = row.to_dict()
        row_dict['source_cd'] = source_cd
        json_job_slug = json.dumps(row_dict)
        list_slugs.append(json_job_slug)

    return list_slugs

row_slugs = get_row_dicts(to_process_df)


In [0]:
row_slugs

In [0]:
#load_configs()

In [0]:
#job_dict = get_widgets()

In [0]:
#job_dict

In [0]:
#get job dict for ASO Monthly
gc = get_gspread_client_from_secret()


source_ss = gc.open_by_url("https://docs.google.com/spreadsheets/d/1WnCBGk1V1nEUeU7aPWBqrQXczmaMop6AbGmkDKkAQcE/edit?gid=1033119277#gid=1033119277")

#

source_wksht = source_ss.worksheet('Validation and Url Tracking')
vals = source_wksht.get_all_values()


headers, data = vals[0],vals[1:]
df = pd.DataFrame(data, columns=headers)

In [0]:
to_process_df = df[df.JobStatus == 'NotStarted']


row_dict = to_process_df.iloc[0].to_dict()
row_dict['SourceCD'] = 'ASO Monthly'

JsonJobSlug = json.dumps(row_dict)


JsonJobSlug

In [0]:
job_dict

In [0]:
#PP Specific validation

def validate_sheet_wksht_format_return_df(workbook_url):
    try:
       ss = gc.open_by_url(workbook_url)
    except:
        raise Exception(f'Unable to open workbook {workbook_url}')

    worksheets = ss.worksheets()
    worksheet_titles = [x.title for x in worksheets]
    if 'input' not in worksheet_titles:
        raise Exception(f"Unable to find 'input' worksheet in workbook {workbook_url}")
    else:
        input_wksht = ss.worksheet('input')
        ### Validate data
        input_values = input_wksht.get_all_values()
        in_header, in_data = input_values[0], input_values[1:]
        
        if len(in_header) != 3:
            raise Exception(f"Invalid number of columns in input worksheet. Expected 3, found {len(in_header)}")

        expected_cols = ['token', 'context', 'en_US']
        for col in in_header:
            if col not in expected_cols:
                raise Exception(f"Invalid column name '{col}' in input worksheet. Expecting columns['token','context','en_US']")
        #print("Num rows in input data:", len(in_data))

        if len(in_data) == 0:
            raise Exception(f"Invalid number of rows in input worksheet. Expected at least 1, found 0")

        init_df = spark.createDataFrame(in_data, ['token','context','en_US'])

    if 'output' not in worksheet_titles:
        print("Not finding 'output' worksheet in workbook {workbook_url}...")
        print("creating 'output' worksheet")
        ss.add_worksheet('output', 400, 50)
        output_wksht = ss.worksheet('output')
        output_wksht.update([['token', 'en_US', *cds]],"A1:Q1")
    if 'summary_stats' not in worksheet_titles:
        print("Not finding 'summary_stats' worksheet in workbook {workbook_url}...")
        print("creating 'summary_stats' worksheet")
        ss.add_worksheet('summary_stats', 50, 50)
        #header_line = ['Metric', *cds]
        summary_stats_sheet = ss.worksheet('summary_stats')
        summary_stats_sheet.update([['Metric', *cds]],"A1:P1")

    return init_df.toPandas()

def get_data_slug(df):
    data_to_print = df.to_dict('records')
    slug = json.dumps(data_to_print, ensure_ascii=False, separators=(",", ":"))
    return slug

In [0]:
%run "./authenticationScript"

In [0]:
gc = get_gspread_client_from_secret()

In [0]:
from config import *
#from language import *
#from translation import *
#from scoring import *

In [0]:
def get_data(job_dict:dict)->pd.DataFrame:

    source_cd = job_dict['source_cd']
    if source_cd == "PP":
       #TODO: prob import this function from a specific PP module
       data = validate_sheet_wksht_format_return_df(job_dict['workbook_url'])
       data_slug = get_data_slug(data)
        
#   if source_cd == "ASO Ad-Hoc":
#        #data = ..
#        #data_slug = ..
#  
#   #if source_cd == "ASO Monthly":
#       #data = ..
#       #data_slug=..
#   
#   #if source_cd =="CS":
#       #data = ...
#       #data_slug = ...


    job_dict['inputs_df'] = data
    job_dict['data_slug']= data_slug

    return job_dict

In [0]:
job_dict = get_data(job_dict)

In [0]:
job_dict

In [0]:
## ASO will be different... 



# CS - will need to get ['network','character_limit','en_US']

In [0]:
# Get Data for PP
#df = validate_sheet_wksht_format_return_df(job_dict['workbook_url'])
# And the Input Slug for PP
#data_slug = get_data_slug(df)

In [0]:
for lang,lang_cd in job_dict['all_language_map'].items():
    print(f"lang: {lang},lang_cd: {lang_cd}")
    #get_PP_prompt(lang, job_dict['data_slug'])

In [0]:
####

In [0]:
# Preproces for ASO Monthly

In [0]:
#Preprocess for ASO AdHoc

In [0]:
#Preprocess for CS

In [0]:
#Preprocess for Other




